# Overview

We will use Recurrent Neural Network (LSTM) and softmax to have a list of label predictions.

Requirements:

- Output file from 1-preprocess-data.ipynb

# Install Dependencies

Our environment will need several ML packages required to import.

## PIP Packages (Optional)

In [6]:
pip install tensorflow numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


## Required Packages

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Hyper Parameters

In [8]:
# Load the CSV
file_path = 'data/output/3-merge-data.csv'
df = pd.read_csv(file_path)

# Tokenize and pad the text data
max_len = 100  # Maximum length of input sequences
vocab_size = 10000  # Vocabulary size

# Training Settings
epochsCount = 4
epochsShuffleData = True

# Split Train and Test Data

In [9]:
# Handle NaN values
df = df.dropna(subset=['singleMessage'])

# Extract features and target
X = df['singleMessage']
y = df['reason']

# Split the dataset into training and testing sets (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len)

# Encode the target labels
label_encoder = LabelEncoder()
label_encoder.fit(y)  # Fit on the entire dataset

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


num_classes = len(label_encoder.classes_)

y_train_categorical = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_categorical = to_categorical(y_test_encoded, num_classes=num_classes)


# Train

We will shuffle our data per each epoch.  We want a list of label probabilities so we will be using softmax activation.

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# Create the RNN model
embedding_dim = 128

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=True),
    LSTM(32),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_padded, y_train_categorical, epochs=epochsCount, validation_data=(X_test_padded, y_test_categorical), shuffle=epochsShuffleData)



Epoch 1/4
2872/2872 [==============================] - 254s 87ms/step - loss: 0.2908 - accuracy: 0.9252 - val_loss: 0.2006 - val_accuracy: 0.9485
Epoch 2/4
2872/2872 [==============================] - 254s 89ms/step - loss: 0.1550 - accuracy: 0.9570 - val_loss: 0.1664 - val_accuracy: 0.9483
Epoch 3/4
2872/2872 [==============================] - 270s 94ms/step - loss: 0.1119 - accuracy: 0.9673 - val_loss: 0.1614 - val_accuracy: 0.9529
Epoch 4/4
2872/2872 [==============================] - 262s 91ms/step - loss: 0.0929 - accuracy: 0.9727 - val_loss: 0.1798 - val_accuracy: 0.9520


# Prediction

Get a list of label probabilities based on the message we provide.

In [29]:
# Example message to predict
new_message = ["TSLA"]

# Preprocess the new message
new_message_seq = tokenizer.texts_to_sequences(new_message)
new_message_padded = pad_sequences(new_message_seq, maxlen=max_len)

# Predict the class probabilities
class_probabilities = model.predict(new_message_padded)

# Sort the class probabilities in descending order along with their corresponding classes
sorted_probabilities = sorted(zip(label_encoder.classes_, class_probabilities[0]), key=lambda x: x[1], reverse=True)

# Print the sorted class probabilities
for reason, probability in sorted_probabilities:
    print(f"{reason}: {probability:.4f}")


1/1 [==============================] - 0s 27ms/step
Clean: 0.9990
Caps for tickers only.: 0.0007
Politics not allowed outside of references to the market.: 0.0001
Off-topic: 0.0001
Inappropriate comment.: 0.0000
Personal or sensitive information not allowed in chat.: 0.0000
"Any discussion related in any way to market manipulation is strictly prohibited, as is advising others on whether to buy, sell, or hold.": 0.0000
Third-party links / content not allowed.: 0.0000
Reviewed by admin internally; not necessary to post to public chat.: 0.0000
False information or no source.: 0.0000
Bullying a member or moderator.: 0.0000
Account number visible. Please remove from content before reposting.: 0.0000
False or misleading information, or no source.: 0.0000
False information.: 0.0000
Bypassing the chat filters is not allowed.: 0.0000
outside link: 0.0000
password: 0.0000
Not sure what this is: 0.0000
language please: 0.0000
comments about market manipulation not allowed even if joking: 0.0000
L

# Save Model
